In [ ]:
#############
#  Modules  #
#############

# Utils
from time import time

# Maths and linear algebra
from numpy import array, sum as npsum

# Knapsack tools
# from knapsack_solvers import *
# from knapsack import kp_qubo, qubo_to_ising_couplings
# from knapsack import kp_instance

# Qiskit
#from qiskit import execute
#from qiskit_aer import StatevectorSimulator
from qiskit_optimization import QuadraticProgram
from qiskit_optimization import QiskitOptimizationError
from qiskit_optimization.algorithms import CplexOptimizer
#from qiskit_optimization.converters import QuadraticProgramToQubo
#from qiskit.circuit import QuantumRegister, QuantumCircuit, Parameter

In [ ]:
#######################
#  Knapsack instance  #
#######################

# Defining the parameters characterizing a KP instance
# n_items = 4
# max_capacity = 4
# kp_profits = array([5, 3, 4, 5])
# kp_weights = array([2, 1, 2, 2])

# Generating KP hard instances
"""
num_instances = 24
n = 10
eps = 1E-5
g = [2, 4, 6, 8]
s = [4, 6, 12, 20, 40, 50]
capacities = [8, 16, 32, 64, 128, 256]
count = 1
for n_class in g:
    for bb, cc in zip(s, capacities):
        kp_instance(
            n_items=n,
            max_capacity=cc,
            classes=n_class,
            fraction=0.1,
            epsilon=eps,
            small=bb,
            instance_id=f"_{count}"
            )
        count+=1
"""

# Loading an instance from the KP instances folder
instance_folder = "kp_instances/small/"
kp_instance_filename = instance_folder + "kp_instance_n_*_C_*"
kp_profits = []
kp_weights = []
with open(kp_instance_filename, 'r') as f:
    n_items = int(f.readline())
    for jj in range(n_items):
        _, pj, wj = f.readline().split()
        kp_profits.append(pj)
        kp_weights.append(wj)
    max_capacity = int(f.readline())
kp_profits = array(kp_profits, dtype=int)
kp_weights = array(kp_weights, dtype=int)

In [ ]:
###########################
#  KP QUBO reformulation  #
###########################
    
# Computing QUBO matrix
# qubo_matrix = kp_qubo(kp_profits, kp_weights, max_capacity, penalty_cte=5)

In [ ]:
##############################
#  KP spinglass Hamiltonian  #
##############################

# Computing spinglass couplings from QUBO
# spinglass_model = qubo_to_ising_couplings(qubo_matrix)

## <span style="color:DarkCyan"> Classical solutions for benchmarking </span>
<br>
In this project, we want to find optimal solutions to the <em>Integer Weight Knapsack Problem (IWKP)</em>, meaning we want to maximize the profit given by the following cost function

$$
\mathcal{V} = \sum_{j=1}^n \ p_j \ x_j
$$

satisfying the following conditions:

$$\begin{align}
& \quad \sum_{j=1}^n \ w_j \ x_j \leq \mathcal{C} \\
& \quad \mathcal{C} > 0, \ \mathcal{C} \in \mathbb{N} \\
& \quad p_j > 0, \ p_j \in \mathbb{N}, \quad \forall j = 1, \dots, n \\
& 0 < w_j < \mathcal{C}, \ w_j \in \mathbb{N} \quad \forall j = 1, \dots, n \\
& \quad \sum_{j=1}^{n} w_j > \mathcal{C} \\
& \quad x_j \in \mathbb{B} \quad \forall j = 1, \dots, n. \\
\end{align}$$

Given that both the cost function and the constraint $(1)$ are linear in the binary optimization variables $\{x_j\}_{j=1}^n$, the problem is mathematically classified as an <em>Binary Integer Linear Program (BILP)</em>. For addressing such problems, we have the option to employ **CPLEX**, a classical solver that is freely accessible via [**Qiskit**](https://docs.quantum.ibm.com/api/qiskit/0.24/qiskit.optimization.algorithms.CplexOptimizer) for relatevely small problem sizes. This BILP solver enables us to derive a classical solution as a benchmark for evaluating the performance of other solvers we plan to develop.

In [ ]:
#############################
#  KP solver: CPlex 4 BLIP  #
#############################

# Creating the ILP as a Quadratic Program for Qiskit CPlex compatibility
## Creating a Quadratic Program instance (QP)
model_name = "_".join([
    "KP",
    f"{n_items:04d}",
    f"{max_capacity:05d}",
    ])
qp_model = QuadraticProgram(model_name)

## Adding the optimization variables 
for jj in range(n_items):
    qp_model.binary_var(f"x_{jj}")
    
## Defining the quadratic cost function (only linear terms in this case)
linear_term_dict = {f'x_{jj}': pj for jj, pj in enumerate(kp_profits)}
qp_model.maximize(linear=linear_term_dict)
    
## Adding the linear inequality constraint (max capacity limit)
lhs_ineq_const = {f'x_{jj}': wj for jj, wj in enumerate(kp_weights)}
cname = f"The total weight of the knapsack must not exceed {max_capacity}"
qp_model.linear_constraint(
    linear=lhs_ineq_const,
    sense="<=",
    rhs=max_capacity,
    name=cname
    )
print("===================================")
print("Knapsack problem instance as a BLIP")
print("===================================")
print(qp_model.prettyprint())
print("===================================")
print("===================================")
print("\n\n")


# Solving the BILP via CPlex
## Solver params
number_of_threads = 1
timelimit = 1e+75
cplex_runtime = 0
cplex_cost = 0
cplex_solution = array([])
cplex_status = ""
cplex_correlation_matrix = array([])

## Checking block
if CplexOptimizer.is_cplex_installed():
    optimizer = CplexOptimizer(
        disp=False,
        cplex_parameters={
            'threads': number_of_threads,
            'randomseed': 0,
            'timelimit': timelimit
            }
        )
else:
    raise Exception("CPlex classical optimizer "
                    "is not available on this machine.")
    
## Solving the QP
try:
    st_time = time()
    cplex_result = optimizer.solve(qp_model)
    et_time = time()
    cplex_runtime = et_time - st_time
    cplex_cost = cplex_result.fval
    cplex_solution = cplex_result.x
    cplex_status = cplex_result.status.name
    cplex_correlation_matrix = cplex_result.get_correlations()
except QiskitOptimizationError:
    status = " ".join([
        "The instantiated quadratic program",
        "is incompatible with the DOcplex optimizer."
        ])
    
## Results
print("=====================================")
print("CPlex classical solution as benchmark")
print("=====================================")
print(f"Classical solution bitstring  -->  {cplex_solution}")
print(f"Total profit  -->  {cplex_cost}")
print(f"Total weight  -->  {int(npsum([wj * xj for wj, xj in zip(kp_weights, cplex_solution)]))}")
print(f"CPU time to classical solution  -->  {cplex_runtime}")
print("=====================================")
print("=====================================")

In [ ]:
############################
#  KP solver: Brute-Force  #
############################

In [ ]:
##############################
#  KP solver: Exact Quantum  #
##############################

# Solver params
# exact_quantum_runtime = 0
# exact_quantum_solution = 0
# exact_quantum_cost = 0

# Ground-state search
## Constructing the Hamiltonian operator
# n_qubits = qubo_matrix.shape[0]
# obs, ops = create_exact_qubit_observables(n_qubits), create_exact_qubit_operators(n_qubits)
# kp_ham = create_exact_hamiltonian_matrix(ops, spinglass_model)

## Finding the ground state
# st_time = time()
# kp_spectrum = find_exact_spectrum(
#     ham_matrix_op=kp_ham,
#     offset=spinglass_model['offset'],
#     renormalized_factor=1,
#     number_of_eigenstates=4
#     )
# et_time = time()
# exact_quantum_runtime = et_time - st_time

## Post-processing the result of the ground-state search
# kp_states = measure_exact_observables(obs, kp_spectrum)
# exact_quantum_solution = array([
#     (sigma + 1) // 2
#     for sigma in kp_states['sz'][0][:n_items]
#     ])
# gs_slack_bitstring = array([
#     (sigma + 1) // 2
#     for sigma in kp_states['sz'][0][n_items:]
#     ])
# exact_quantum_cost = -kp_spectrum[0][0]
# slack_opt_value = npsum([
    # 2**q * bq
    # for q, bq in enumerate(gs_slack_bitstring)
    # ])

## Results
# print("======================")
# print("Exact quantum solution")
# print("======================")
# print(f"Exact quantum bitstring  -->  {exact_quantum_solution}")
# print(f"Total profit  -->  {exact_quantum_cost}")
# print(f"Total weight  -->  {int(npsum([wj * xj for wj, xj in zip(kp_weights, exact_quantum_solution)]))}")
# print(f"Slack integer variable optimal cost  -->  {slack_opt_value}")
# print(f"CPU time to classical solution  -->  {exact_quantum_runtime}")
# print("=====================================")
# print("=====================================")